In [2]:
import lmstudio as lms

downloaded = lms.list_downloaded_models()
llm_only = lms.list_downloaded_models("llm")
embedding_only = lms.list_downloaded_models("embedding")

for model in downloaded:
    print(model.model_key)

text-embedding-nomic-embed-text-v1.5
internvl3-8b
smolvlm2-2.2b-instruct
irix-12b-model_stock
deepseek-r1-0528-qwen3-8b
qwen2.5-vl-32b-instruct
qwen2.5-vl-7b-instruct
gemma-3-4b-it
qwen2-vl-2b-instruct
granite-vision-3.2-2b
smolvlm2-500m-video-instruct
gemma-3-27b-it-qat
gemma-3-12b-it-qat


In [3]:
model = lms.llm('gemma-3-4b-it')
print(model.respond("What is the meaning of life?"))

Okay, let's tackle this big one – the meaning of life. The short answer is: **there isn’t a single, universally agreed-upon answer.** It's arguably *the* most pondered question in human history, and that's because it's deeply personal and philosophical. 

Here's a breakdown of different perspectives, grouped into categories:

**1. Philosophical Perspectives:**

* **Nihilism:** This view suggests life is inherently meaningless. There’s no objective purpose or value. It can be bleak, but some find freedom in accepting this lack of inherent meaning and creating their own.
* **Existentialism:**  This philosophy argues that existence precedes essence. We are born into the world without a predetermined purpose. *We* create our own meaning through our choices and actions. Key figures include Sartre and Camus. It emphasizes responsibility – we're responsible for defining ourselves.
* **Absurdism:** Closely related to existentialism, absurdism recognizes the conflict between humanity’s innate d

In [4]:
from datasets import load_dataset

ds = load_dataset("openai/gsm8k", "main")

C:\Users\CoolA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\CoolA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\CoolA\.cache\huggingface\hub\datasets--openai--gsm8k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/h

In [ ]:
import pandas as pd
import lmstudio as lms
from datasets import load_dataset
import json
import random

# Load dataset and prerequisites
ds = load_dataset("openai/gsm8k", "main")
concepts_df = pd.read_csv("D:\\eduvlm-bench\\cps\\project-lambda\\gsm8k_unique_concepts.csv")
unique_concepts = concepts_df['Concept'].tolist()

# Load model
model = lms.llm('gemma-3-4b-it')

def identify_prerequisites_for_question(question, concepts_list):
    """
    Use LLM to identify which prerequisites are needed for a specific question
    """
    prompt = f"""
Given this math word problem and a list of mathematical concepts, identify which concepts are ESSENTIAL prerequisites to solve this problem.

Problem: {question}

Available Concepts:
{', '.join(concepts_list)}

Instructions:
- Only list concepts that are absolutely necessary to solve this problem
- Be precise - don't include concepts that are merely related but not required
- Return only the concept names, separated by commas
- If none of the listed concepts are needed, return "None"

Essential Prerequisites:"""
    
    response = model.respond(prompt)
    # Parse the response to extract concept names
    prerequisites = [concept.strip() for concept in response.content.split(',') if concept.strip() in concepts_list]
    return prerequisites

def generate_wrong_answer_with_missing_prerequisite(question, missing_concept):
    """
    Generate wrong answer by having model pretend it doesn't know a specific concept
    """
    prompt = f"""
You are solving a math word problem, but you have a gap in your mathematical knowledge - you don't understand or remember how to work with "{missing_concept}".

Problem: {question}

Instructions:
- Attempt to solve this problem step by step
- When you encounter the concept "{missing_concept}", make a plausible mistake or use incorrect reasoning
- Show your work clearly
- Your final answer should be wrong due to the missing knowledge
- Make the mistake realistic - something a student might actually do

Your solution:"""
    
    response = model.respond(prompt)
    return response

# Main processing loop for first 200 questions
results = []

for i in range(min(200, len(ds['train']))):
    question = ds['train'][i]['question']
    correct_answer = ds['train'][i]['answer']
    
    print(f"Processing question {i+1}/200...")
    
    # Step 1: Identify prerequisites for this specific question
    prerequisites = identify_prerequisites_for_question(question, unique_concepts)
    
    if not prerequisites:
        print(f"No prerequisites identified for question {i+1}, skipping...")
        continue
    
    # Step 2: Randomly select one prerequisite to remove
    missing_concept = random.choice(prerequisites)
    
    # Step 3: Generate wrong answer with missing prerequisite
    wrong_answer = generate_wrong_answer_with_missing_prerequisite(question, missing_concept)
    wrong_answer = wrong_answer.content
    
    # Store results
    result = {
        'question_id': i,
        'question': question,
        'correct_answer': correct_answer,
        'all_prerequisites': prerequisites,
        'missing_prerequisite': missing_concept,
        'wrong_answer': wrong_answer
    }
    
    results.append(result)
    
    # Save intermediate results every 10 questions
    if (i + 1) % 10 == 0:
        with open(f'gsm8k_wrong_answers_checkpoint_{i+1}.json', 'w') as f:
            json.dump(results, f, indent=2)

# Save final results
results_df = pd.DataFrame(results)
results_df.to_csv('gsm8k_wrong_answers_with_missing_prerequisites.csv', index=False)

print(f"Completed processing {len(results)} questions!")
print("Results saved to 'gsm8k_wrong_answers_with_missing_prerequisites.csv'")

Processing question 1/200...
Processing question 2/200...
Processing question 3/200...
Processing question 4/200...
Processing question 5/200...
Processing question 6/200...
Processing question 7/200...
Processing question 8/200...
Processing question 9/200...
Processing question 10/200...
Processing question 11/200...
Processing question 12/200...
Processing question 13/200...
Processing question 14/200...
Processing question 15/200...
Processing question 16/200...
Processing question 17/200...
Processing question 18/200...
Processing question 19/200...
Processing question 20/200...
Processing question 21/200...
Processing question 22/200...
Processing question 23/200...
Processing question 24/200...
Processing question 25/200...
Processing question 26/200...
Processing question 27/200...
Processing question 28/200...
Processing question 29/200...
Processing question 30/200...
Processing question 31/200...
Processing question 32/200...
Processing question 33/200...
Processing question